In [15]:
import os
import time
import mxnet as mx
import numpy as np
from math import erf, sqrt
import cv2
import boto3

print('intialize mxnet: {}, np: {}, cv2: {}'.format(mx.__version__, np.__version__, cv2.__version__))

class Scorer():

    def __init__(self, model_dir, image_size=(112,112)):
        # Load the mobilenet face model
        self.model_dir = model_dir
        self.image_size = image_size
        model_start = time.time()
        # Load the mxnet model
        model_str = os.path.join(self.model_dir, 'mobilenet1,0')
        print('loading face model {}, size: {}'.format(model_str, self.image_size))
        self.model = self.get_model(mx.cpu(), self.image_size, model_str, 'fc1')
        print('face model loaded in {}'.format(time.time()-model_start))
        # Load the people model
        self.people_path = os.path.join(self.model_dir, 'people.npz')
        self.load_model()

    def load_model(self):
        # Load people database, if we can't find or an error, create new db
        people_start = time.time()
        try:
            print('loading face db {}'.format(self.people_path))
            people_db = np.load(self.people_path)
            self.vecs = people_db['vecs']
            self.names = people_db['names']
        except FileNotFoundError as e:
            print('initialize face db', e)
            self.clear_model()
        print('face db loaded in {}s, vecs:{}'.format(time.time()-people_start, self.vecs.shape))

    def clear_model(self):
        self.vecs = np.zeros((0,128), dtype=float)
        self.names = np.zeros((0), dtype=str)

    def get_model(self, ctx, image_size, model_str, layer):
        _vec = model_str.split(',')
        assert len(_vec)==2
        prefix = _vec[0]
        epoch = int(_vec[1])
        sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
        all_layers = sym.get_internals()
        sym = all_layers[layer+'_output']
        model = mx.mod.Module(symbol=sym, context=ctx, label_names = None)
        #model.bind(data_shapes=[('data', (args.batch_size, 3, image_size[0], image_size[1]))], label_shapes=[('softmax_label', (args.batch_size,))])
        model.bind(data_shapes=[('data', (1, 3, image_size[0], image_size[1]))])
        model.set_params(arg_params, aux_params)
        return model

    def get_input(self, image_size, img, bbox=None, rotate=0, margin=44):
        # Call preprocess() to generate aligned images
        if bbox is None:
            det = np.zeros(4, dtype=np.int32)
            det[0] = int(img.shape[1]*0.0625)
            det[1] = int(img.shape[0]*0.0625)
            det[2] = img.shape[1] - det[0]
            det[3] = img.shape[0] - det[1]
        else:
            det = bbox
        # Crop
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2, img.shape[1])
        bb[3] = np.minimum(det[3]+margin/2, img.shape[0])
        img = img[bb[1]:bb[3],bb[0]:bb[2],:]
        # Rotate if required
        if 0 < rotate and rotate < 360:
            rows,cols,_ = img.shape
            M = cv2.getRotationMatrix2D((cols/2,rows/2),360-rotate,1)
            img = cv2.warpAffine(img,M,(cols,rows))
        # Resize and transform
        img = cv2.resize(img, (image_size[1], image_size[0]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        aligned = np.transpose(img, (2,0,1))
        return aligned

    def get_feature(self, model, aligned):
        def l2_normalize(X):
            norms = np.sqrt((X * X).sum(axis=1))
            X /= norms[:, np.newaxis]
            return X
        input_blob = np.expand_dims(aligned, axis=0)
        data = mx.nd.array(input_blob)
        db = mx.io.DataBatch(data=(data,))
        model.forward(db, is_train=False)
        embedding = model.get_outputs()[0].asnumpy()
        embedding = l2_normalize(embedding).flatten()
        return embedding

    def vectorize(self, img, bbox=None, rotate=0, margin=0):
        aligned = self.get_input(self.image_size, img, bbox=bbox, rotate=rotate, margin=margin)
        vec = self.get_feature(self.model, aligned)
        return aligned, vec

    def similar(self, vec):
        def phi(x):
            #'Cumulative distribution function for the standard normal distribution'
            return (1.0 + erf(x / sqrt(2.0))) / 2.0
        assert self.vecs.shape[1]==vec.shape[0]
        sims = np.dot(self.vecs, vec)
        sim_idx = np.argmax(sims)
        sim = sims[sim_idx]
        z_score = (sim - sims.mean()) / sims.std()
        return sim, z_score, phi(z_score), self.names[sim_idx]


intialize mxnet: 1.5.0, np: 1.16.4, cv2: 3.4.2


##  Create model bucket

Creat model bucket and upload models and people.npz

In [26]:
import boto3

region_name = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
model_bucket = 'deeplens-model-{}-{}'.format(account_id, region_name)

!aws s3 mb s3://$model_bucket

make_bucket: deeplens-model-423079281568-ap-southeast-2


In [27]:
!aws s3 cp models/mobilenet1-0000.params s3://$model_bucket/mobilenet1/
!aws s3 cp models/mobilenet1-symbol.json s3://$model_bucket/mobilenet1/
!aws s3 cp people.npz s3://$model_bucket/mobilenet1/

upload: models/mobilenet1-0000.params to s3://deeplens-model-423079281568-ap-southeast-2/mobilenet1/mobilenet1-0000.params
upload: models/mobilenet1-symbol.json to s3://deeplens-model-423079281568-ap-southeast-2/mobilenet1/mobilenet1-symbol.json
upload: ./people.npz to s3://deeplens-model-423079281568-ap-southeast-2/mobilenet1/people.npz


### Download and score

Test downloading to an artifacts folder and scoring

In [36]:
!aws s3 sync s3://$model_bucket/mobilenet1 ./artifacts

In [29]:
# Load model
model_start = time.time()
scorer = Scorer('./artifacts/')
msg = 'Image classification model loaded {} in {}s'.format(scorer.vecs.shape[0], time.time()-model_start)
sim_threshold = 0.95

loading face model ./artifacts/mobilenet1,0, size: (112, 112)
face model loaded in 0.2219395637512207
loading face db ./artifacts/people.npz
face db loaded in 0.007072925567626953s, vecs:(226, 128)


Download sample image

In [30]:
!aws s3 cp s3://aiml-lab-sagemaker/politicians/politicians1.jpg tmp/image

download: s3://aiml-lab-sagemaker/politicians/politicians1.jpg to tmp/image


Get the image frame and bbox size

In [33]:
# Call rekognition to get the bbox dims
rekognition = boto3.client('rekognition')

def get_bbox(img):
    # Detect faces
    ret, buf = cv2.imencode('.jpg', img)
    ret = rekognition.detect_faces(
        Image={
            'Bytes': buf.tobytes()
        },
        Attributes=['DEFAULT'],
    )
    # Return the bounding boxes for each face
    height, width, _ = img.shape
    bboxes = []
    for face in ret['FaceDetails']:
        box = face['BoundingBox']
        x1 = int(box['Left'] * width)
        y1 = int(box['Top'] * height)
        x2 = int(box['Left'] * width + box['Width'] * width)
        y2 = int(box['Top'] * height + box['Height']  * height)
        return [x1, y1, x2, y2]

# Load sample image and get bbox
frame = cv2.imread('tmp/image')
[xmin,ymin,xmax,ymax] = get_bbox(frame)

Find most similar image

In [35]:
# Vectorize 
bbox = [xmin,ymin,xmax,ymax]
aligned, vec = scorer.vectorize(frame, bbox)
sim, z_score, prob, name = scorer.similar(vec)

print('sim: {:.4f}, zscore: {:.4f}, prob: {:.4f}, name: {}'.format(sim, z_score, prob, name))

sim: 0.4492, zscore: 2.5082, prob: 0.9939, name: Hon Michael McCormack MP
